<a href="https://colab.research.google.com/github/BethanyJep/100DaysOfMlCode/blob/master/sreamjobs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Getting the necessary libraries
import tweepy as tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import time
import json
import pandas as pd
import matplotlib.pyplot as plt

import csv
import re #regular expression
from textblob import TextBlob
import string

In [2]:
#consumer keys
consumer_key = "fslgtxGRaV0NooQSyV6bMO8JK"
consumer_secret = "CQLaOZU9dhT2AALorcu2KxC9rfNpC4ifu0QvtwnDUlHvrEjTxE"
access_token = "924336306481754113-J80n52buzVcvRag0bGI1jUVmaE8TxxY"
access_token_secret = "jnAEuQlSXlFnilRbtiPMr1inhRhVMihBDZFNBeJDExqrE"

In [3]:
from tweepy import API
from tweepy import OAuthHandler

# Consumer key authentication
auth = OAuthHandler(consumer_key, consumer_secret)

# Access key authentication
auth.set_access_token(access_token, access_token_secret)

# Set up the API with the authentication handler
api = API(auth)

In [4]:
from tweepy.streaming import StreamListener
import json
import time
import sys

class SListener(StreamListener):
    def __init__(self, api = None, fprefix = 'streamer'):
        self.api = api or API()
        self.counter = 0
        self.fprefix = fprefix
        self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')


    def on_data(self, data):
        if  'in_reply_to_status' in data:
            self.on_status(data)
        elif 'delete' in data:
            delete = json.loads(data)['delete']['status']
            if self.on_delete(delete['id'], delete['user_id']) is False:
                return False
        elif 'limit' in data:
            if self.on_limit(json.loads(data)['limit']['track']) is False:
                return False
        elif 'warning' in data:
            warning = json.loads(data)['warnings']
            print("WARNING: %s" % warning['message'])
            return


    def on_status(self, status):
        self.output.write(status)
        self.counter += 1
        if self.counter >= 20000:
            self.output.close()
            self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')
            self.counter = 0
        return


    def on_delete(self, status_id, user_id):
        print("Delete notice")
        return


    def on_limit(self, track):
        print("WARNING: Limitation notice received, tweets missed: %d" % track)
        return


    def on_error(self, status_code):
        print('Encountered error with status code:', status_code)
        return 


    def on_timeout(self):
        print("Timeout, sleeping for 60 seconds...")
        time.sleep(60)
        return 

In [7]:
# Set up words to track
keywords_to_track = ["#IkoKaziKE"]
#, '#ikokazi', 'Iko Kazi KE','#ikokazike', '#PataKaziKE', 'my next client might be on your TL', '#KOT', '#KenyansOnTwitter', '#ShareMyHustle', '+254', '07', 'hiring', 'We are hiring '
# Instantiate the SListener object 
listen = SListener(api)

# Instantiate the Stream object
stream = Stream(auth, listen)

# Begin collecting data
stream.filter(track = keywords_to_track, async=True)

Encountered error with status code: 420
